Постройте модель, используя архитектуру со свёрточными слоями, для классификации Fashion MNIST. Итоговое качество (accuracy) должно быть не ниже 89,5.

In [ ]:
import torch
import torchvision as tv
from torchsummary import summary
import time

In [ ]:
train_ds = tv.datasets.FashionMNIST('.',
                                    train=True,
                                    transform=tv.transforms.ToTensor(),
                                    download=True)

test_ds = tv.datasets.FashionMNIST('.',
                                  train=False,
                                  transform=tv.transforms.ToTensor(),
                                  download=True)

100%|██████████| 26421880/26421880 [00:03<00:00, 7682427.94it/s] 


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 143235.03it/s]


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2663935.86it/s]


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6581004.87it/s]

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [ ]:
batch_size = 128
train = torch.utils.data.DataLoader(train_ds, batch_size=batch_size)
test = torch.utils.data.DataLoader(train_ds, batch_size=batch_size)

In [ ]:
model = torch.nn.Sequential(
  torch.nn.Conv2d(1, 32, kernel_size=3),
  torch.nn.BatchNorm2d(32),
  torch.nn.ReLU(),
  torch.nn.MaxPool2d(2),
  torch.nn.Dropout2d(0.2),
  torch.nn.Conv2d(32, 64, kernel_size=3),
  torch.nn.BatchNorm2d(64),
  torch.nn.ReLU(),
  torch.nn.MaxPool2d(2),
  torch.nn.Dropout2d(0.2),
  torch.nn.Conv2d(64, 256, kernel_size=3),
  torch.nn.BatchNorm2d(256),
  torch.nn.ReLU(),
  torch.nn.MaxPool2d(2),
  torch.nn.Dropout2d(0.2),
  torch.nn.Flatten(),
  torch.nn.Linear(256, 128),
  torch.nn.ReLU(),
  torch.nn.Linear(128, 10)
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model = model.to(device)

In [ ]:
summary(model=model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
       BatchNorm2d-2           [-1, 32, 26, 26]              64
              ReLU-3           [-1, 32, 26, 26]               0
         MaxPool2d-4           [-1, 32, 13, 13]               0
         Dropout2d-5           [-1, 32, 13, 13]               0
            Conv2d-6           [-1, 64, 11, 11]          18,496
       BatchNorm2d-7           [-1, 64, 11, 11]             128
              ReLU-8           [-1, 64, 11, 11]               0
         MaxPool2d-9             [-1, 64, 5, 5]               0
        Dropout2d-10             [-1, 64, 5, 5]               0
           Conv2d-11            [-1, 256, 3, 3]         147,712
      BatchNorm2d-12            [-1, 256, 3, 3]             512
             ReLU-13            [-1, 256, 3, 3]               0
        MaxPool2d-14            [-1, 25

In [ ]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(epochs, batch_size, model, trainer, loss, train_ds, test_ds):

  for ep in range(epochs):

    train_iter, train_pass = 0, 0
    train_loss, train_acc = 0., 0.

    start = time.time()

    model.train()

    for X, y in train_ds:
      X, y = X.to(device), y.to(device)
      trainer.zero_grad()
      y_pred = model(X)
      l = loss(y_pred, y)
      l.backward()
      trainer.step()
      train_loss += l.item()
      train_acc += (y_pred.argmax(dim=1) == y).sum().item()
      train_iter += 1
      train_pass += len(X)

    test_iter, test_pass  = 0, 0
    test_loss, test_acc = 0., 0.

    model.eval()

    for X, y in test_ds:
      X, y = X.to(device), y.to(device)
      y_pred = model(X)
      l = loss(y_pred, y)
      test_loss += l.item()
      test_acc += (y_pred.argmax(dim=1) == y).sum().item()
      test_iter += 1
      test_pass += len(X)

    print(f"epoch: {ep}",
          f"time: {time.time() - start}",
          f"train_loss: {train_loss / train_iter}",
          f"train_acc: {train_acc / train_pass}",
          f"test_loss: {test_loss / test_iter}",
          f"test_acc: {test_acc / test_pass}")

In [ ]:
train_model(epochs=10, batch_size=batch_size, model=model, trainer=trainer, loss=loss, train_ds=train, test_ds=test)

epoch: 0 time: 19.621450662612915 train_loss: 0.20638627398497006 train_acc: 0.9226 test_loss: 0.13968900502172868 test_acc: 0.94645
epoch: 1 time: 18.927807807922363 train_loss: 0.20246334737742633 train_acc: 0.9235333333333333 test_loss: 0.13701302894174672 test_acc: 0.9483333333333334
epoch: 2 time: 20.349509954452515 train_loss: 0.1994731322661646 train_acc: 0.9242333333333334 test_loss: 0.133402945366559 test_acc: 0.9492833333333334
epoch: 3 time: 19.868161916732788 train_loss: 0.19505740627487586 train_acc: 0.9265 test_loss: 0.1279456320443133 test_acc: 0.95195
epoch: 4 time: 18.951786756515503 train_loss: 0.18980045961355096 train_acc: 0.9272666666666667 test_loss: 0.1323106326798259 test_acc: 0.9494666666666667
epoch: 5 time: 19.49898934364319 train_loss: 0.18811474392599642 train_acc: 0.9287166666666666 test_loss: 0.12187795900999865 test_acc: 0.95465
epoch: 6 time: 18.90048384666443 train_loss: 0.18687237661752873 train_acc: 0.9281666666666667 test_loss: 0.12858419943211682 t